<a href="https://colab.research.google.com/github/greeshmanth-5/Greeshmanth_INFO5731_Fall2023/blob/main/In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [3]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

# Sample data
sample_data = [
    "This movie is absolutely fantastic! I loved every minute of it.",
    "The acting was terrible, and the plot made no sense.",
    "I'm not sure how to feel about this film. It had its moments, but overall, it was mediocre.",
]

# Preprocess the sample data
data_words = list(sent_to_words(sample_data))

# Remove stop words
data_words = remove_stopwords(data_words)

import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

from pprint import pprint

# Number of topics
num_topics = 2  # You can adjust the number of topics as needed

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                         id2word=id2word,
                                         num_topics=num_topics)

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(0,
  '0.070*"minute" + 0.070*"absolutely" + 0.070*"fantastic" + 0.069*"every" + '
  '0.068*"movie" + 0.066*"loved" + 0.057*"moments" + 0.057*"film" + '
  '0.056*"feel" + 0.056*"mediocre"'),
 (1,
  '0.069*"sense" + 0.066*"made" + 0.065*"acting" + 0.065*"terrible" + '
  '0.064*"plot" + 0.062*"overall" + 0.062*"sure" + 0.061*"mediocre" + '
  '0.061*"feel" + 0.061*"film"')]


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [6]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text
from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Sample data
sample_data = [
    "This movie is absolutely fantastic! I loved every minute of it.",
    "The acting was terrible, and the plot made no sense.",
    "I'm not sure how to feel about this film. It had its moments, but overall, it was mediocre.",
]

# Create a DataFrame for the sample data
df = pd.DataFrame({'Review': sample_data})

# Preprocess the text using the same function
def preprocess(text):
    CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_punctuation, strip_short, stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    return text

# Apply the preprocessing to the sample data
df['Review_Text (Clean)'] = df['Review'].apply(lambda x: preprocess(x))

# Create a dictionary with the corpus
corpus = df['Review_Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# Convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

# Coherence score in topic modeling to measure how interpretable the topics are to humans.
# Find the coherence score with a different number of topics
for i in range(2, 11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Review_Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

# Perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)

# Find the 5 words with the strongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

# Find the scores given between the review and each topic
corpus


Coherence score with 2 clusters: 0.2974621508745988
Coherence score with 3 clusters: 0.29746215087459865
Coherence score with 4 clusters: 0.29746215087459865
Coherence score with 5 clusters: 0.29746215087459876
Coherence score with 6 clusters: 0.2974621508745989
Coherence score with 7 clusters: 0.29746215087459865
Coherence score with 8 clusters: 0.2974621508745987
Coherence score with 9 clusters: 0.2974621508745988
Coherence score with 10 clusters: 0.2974621508745987
Words in 0: 0.408*"film" + 0.408*"feel" + 0.408*"sure" + 0.408*"overal" + 0.408*"moment" + 0.408*"mediocr" + 0.000*"absolut" + -0.000*"fantast" + -0.000*"love" + -0.000*"minut".
Words in 1: 0.447*"fantast" + 0.447*"absolut" + 0.447*"minut" + 0.447*"movi" + 0.447*"love" + 0.000*"feel" + 0.000*"sure" + 0.000*"film" + 0.000*"moment" + 0.000*"overal".


0          [movi, absolut, fantast, love, minut]
1                     [act, terribl, plot, sens]
2    [sure, feel, film, moment, overal, mediocr]
Name: Review_Text (Clean), dtype: object

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [7]:
import nltk
nltk.download('all')
!pip install preprocess
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [8]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [10]:
import pyLDAvis
pyLDAvis.enable_notebook()
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Sample data
sample_data = [
    "This movie is absolutely fantastic! I loved every minute of it.",
    "The acting was terrible, and the plot made no sense.",
    "I'm not sure how to feel about this film. It had its moments, but overall, it was mediocre.",
]

# Preprocess the sample data
sample_data_clean = [preprocess(text) for text in sample_data]

# Join the preprocessed texts into a list of strings
sample_data_clean_text = [' '.join(text) for text in sample_data_clean]

# Create a CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_data_clean_text)

# Get the top words for each topic
top = 10
topic_to_topwords = {}
for j in range(X.shape[0]):
    top_words_indices = np.argsort(X[j].toarray()[0])[::-1][:top]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_words_indices]
    msg = 'Topic %i has top words: %s' % (j, ', '.join(top_words))
    print(msg)
    topic_to_topwords[j] = top_words




Topic 0 has top words: movi, minut, love, fantast, absolut, terribl, sure, sens, plot, overal
Topic 1 has top words: terribl, sens, plot, act, sure, overal, movi, moment, minut, mediocr
Topic 2 has top words: sure, overal, moment, mediocr, film, feel, terribl, sens, plot, movi


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [34]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bertopic import BERTopic
import matplotlib.pyplot as plt

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to clean text
stop_words = set(stopwords.words('english'))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:



from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all')['data']

topic_model = BERTopic(nr_topics="auto", calculate_probabilities=True, verbose=True)
topics, _ = topic_model.fit_transform(data)

topic_overview = topic_model.get_topic_freq()

for topic_num, freq in topic_overview[1:].values:
    topic_words = topic_model.get_topic(topic_num)
    topic_summary = ", ".join([word[0] for word in topic_words[:5]])
    print(f"Topic {topic_num}: {topic_summary} (Freq: {freq})")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2023-11-06 05:47:34,706 - BERTopic - Transformed documents to Embeddings
2023-11-06 05:48:26,172 - BERTopic - Reduced dimensionality
2023-11-06 05:51:00,415 - BERTopic - Clustered reduced embeddings
2023-11-06 05:51:27,164 - BERTopic - Reduced number of topics from 379 to 247


Topic 0: game, team, games, players, he (Freq: 1063)
Topic 1: gun, the, president, that, to (Freq: 992)
Topic 2: god, jesus, faith, atheists, that (Freq: 579)
Topic 3: car, cars, ford, engine, mustang (Freq: 474)
Topic 4: israel, israeli, jews, arab, jewish (Freq: 414)
Topic 5: jpeg, image, gif, format, images (Freq: 300)
Topic 6: armenian, turkish, armenians, were, armenia (Freq: 291)
Topic 7: homosexual, homosexuality, gay, homosexuals, men (Freq: 220)
Topic 8: mail, graphics, send, image, ray (Freq: 164)
Topic 9: monitor, monitors, vga, nec, nanao (Freq: 155)
Topic 10: printer, deskjet, hp, printers, laser (Freq: 148)
Topic 11: modem, modems, serial, fax, courier (Freq: 147)
Topic 12: amp, sale, sony, stereo, speakers (Freq: 146)
Topic 13: colormap, color, bit, xv, cursor (Freq: 145)
Topic 14: drivers, card, diamond, driver, ati (Freq: 124)
Topic 15: cancer, lyme, patients, doctor, medical (Freq: 118)
Topic 16: dos, windows, memory, allocation, emm386 (Freq: 111)
Topic 17: microsoft

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

-->To compare topic modeling algorithms (LDA, NMF, LSA, and BERTopic),we have to consider
factors such as topic quality and interpretability.

-->LDA provides highly interpretable topics with word probability distributions.

-->NMF and LSA offer reasonably interpretable topics but may differ from LDA.

-->BERTopic can produce semantically meaningful topics but may differ from traditional methods due to its BERT model dependency.

-->Evaluate topic coherence; LDA and NMF often have good coherence.

-->LSA tends to have lower topic coherence.

-->BERTopic can have good coherence, depending on the chosen BERT model.

-->Consider scalability; LDA, NMF, and LSA are scalable to large datasets.

-->BERTopic can be computationally intensive, especially with large BERT models.

-->Assess hyperparameter sensitivity and model robustness for your project, and consider the availability of pre-trained models.

-->Because of its explicit word probability distributions for each topic, LDA is frequently regarded as the best solution for interpretability.

-->NMF and LSA can be good solutions if you want good interpretability with better scalability.

-->BERTopic provides semantic context, but its interpretability varies and it may be more computationally intensive.

Because of the aforementioned characteristics and by comparing results, I believe LSA is the most efficient of the four models.
